In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import itertools

### Read in CSV file:

In [2]:
# Load in 60 minutes:
df_60 = pd.read_csv('../backtesting/backcast_csv_data/ETHUSD_60.csv', header=None, names=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'trades'])
df_60['date'] = pd.to_datetime(df_60['timestamp'], unit='s')
df_60.head()

,timestamp,open,high,low,close,volume,trades,date
0,1438956000,3.00000,3.00000,3.00,3.00,81.857278,2,2015-08-07 14:00:00
1,1438966800,3.00001,3.00001,3.00,3.00,42.073291,2,2015-08-07 17:00:00
2,1439013600,3.00000,3.00000,3.00,3.00,0.400000,1,2015-08-08 06:00:00
3,1439024400,2.00000,2.00000,2.00,2.00,125.000000,2,2015-08-08 09:00:00
4,1439049600,1.20000,1.20000,1.02,1.02,234.445300,5,2015-08-08 16:00:00


## Question to Answer:

What combination of: 

1. Interval aggregate (1 hour, 4 hours, 12 hours, 1 day, etc)
2. Hold window (3 periods, 4 periods, etc)

produces the largest win ratio?

### Percent Change Analysis - 1 Hour

In [3]:
epoch= pd.Timestamp("2015-12-27")  
df_60["week_number"]=np.where(df_60.date.astype("datetime64").le(epoch), df_60.date.dt.isocalendar().week, df_60.date.sub(epoch).dt.days//7+52)

In [46]:
interval_aggs = [60, 120, 240, 360, 720, 1440]
hold_windows = [3, 4, 5, 6, 7]


for agg,hold in itertools.product(interval_aggs, hold_windows):
    
    # Re-aggregate to fit interval:
    df = df_60.resample(f'{agg}T').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum',
    'trades': 'sum'
        })
    df = df.ffill()
    
    df[f'pct_change_{str(hold)}'] = df['close'].pct_change(hold)
    df[f'pct_change_{str(hold)}_net'] = [1 if x > 0 else 0 for x in df[f'pct_change_{str(hold)}']] 
    
    hold_time_calc = agg*hold
    if hold_time_calc >= 1440:
        hold_time = hold_time_calc/1440
        print_string = f'{hold_time} days'
    else:
        hold_time = hold_time_calc/60
        print_string = f'{hold_time} hours'
    
    # Print results:
    print(f'Results of {int(agg/60)} hour(s) candlesticks holding for {print_string}:')
    
    ## Win percent:
    win_pct = round(df[f'pct_change_{str(hold)}_net'].sum()/df[f'pct_change_{str(hold)}_net'].count()*100, 2)
    print(f'Win percent: {win_pct}%')
    
    ## Average win:
    df_win = df[df[f'pct_change_{str(hold)}_net'] == 1]
    avg_win = round(df_win[f'pct_change_{str(hold)}'].mean()*100, 2)
    print(f'Average Win: {avg_win}%')
    
    ## Avergae loss:
    df_loss = df[df[f'pct_change_{str(hold)}_net'] == 0]
    avg_loss = round(df_loss[f'pct_change_{str(hold)}'].mean()*100, 2)
    print(f'Average Loss: {avg_loss}%')
    
    print('\n')
    

Results of 1 hour(s) candlesticks holding for 3.0 hours:
Win percent: 48.67%
Average Win: 1.63%
Average Loss: -1.32%


Results of 1 hour(s) candlesticks holding for 4.0 hours:
Win percent: 49.24%
Average Win: 1.89%
Average Loss: -1.54%


Results of 1 hour(s) candlesticks holding for 5.0 hours:
Win percent: 49.43%
Average Win: 2.12%
Average Loss: -1.73%


Results of 1 hour(s) candlesticks holding for 6.0 hours:
Win percent: 49.76%
Average Win: 2.31%
Average Loss: -1.9%


Results of 1 hour(s) candlesticks holding for 7.0 hours:
Win percent: 50.12%
Average Win: 2.49%
Average Loss: -2.07%


Results of 2 hour(s) candlesticks holding for 6.0 hours:
Win percent: 49.8%
Average Win: 2.31%
Average Loss: -1.91%


Results of 2 hour(s) candlesticks holding for 8.0 hours:
Win percent: 50.38%
Average Win: 2.66%
Average Loss: -2.23%


Results of 2 hour(s) candlesticks holding for 10.0 hours:
Win percent: 50.55%
Average Win: 2.99%
Average Loss: -2.49%


Results of 2 hour(s) candlesticks holding for 12.